In [11]:
from typing import List
from src.data_type import Measurement
import json

In [12]:
measurements = [
  Measurement(5, 1000),
  Measurement(3, 1100),
  Measurement(2, 1200),
]

In [16]:
from senml import senml

In [23]:
json_object = json.loads('''
[
{
    "bn":"urn:dev:mac:0b92569229fc9e68/rpm/",
    "bt":1234567890.123,
    "bu":"1/min",
    "bver":5,
    "n":"fwd",
    "v":17.666544,
    "s":3,
    "t":5
},
{
    "n":"rev",
    "v":123.456,
    "s":4,
    "t":7
},
{
    "n":"bool",
    "vb":true,
    "t":1
},
{
    "n":"str",
    "vs":"hej",
    "t":2
}
]
''')
doc = senml.SenMLDocument.from_json(json_object)
doc
senml.SenMLDocument.to_json(doc)

[{'bver': 5,
  'bn': 'urn:dev:mac:0b92569229fc9e68/rpm/',
  'bt': 1234567890.123,
  'bu': '1/min',
  'n': 'fwd',
  't': 5.0,
  's': 3.0,
  'v': 17.666544},
 {'n': 'rev', 't': 7.0, 's': 4.0, 'v': 123.456},
 {'n': 'bool', 't': 1.0, 'vb': True},
 {'n': 'str', 't': 2.0, 'vs': 'hej'}]

In [39]:
from kpn_senml import *
import random
import cbor2

pack = SenmlPack('/72')
pack.add(SenmlRecord("1/2", value=random.randint(0,1000), time=1))
cbor_val = pack.to_cbor()
print(cbor_val)
print(cbor2.loads(cbor_val))

b'\x81\xa4\x00c1/2\x02\x19\x01\xfa\x06\x01!c/72'
[{0: '1/2', 2: 506, 6: 1, -2: '/72'}]


In [13]:
class CBOR_Serializer:
  def serialize(measurements: List[Measurement], a: int, b: int, c: int):
    output = bytearray()
    output.append(0xA1)
    return output

cbor_parser = CBOR_Serializer()
print(cbor_parser.serialize(measurements))

TypeError: serialize() missing 2 required positional arguments: 'b' and 'c'

In [ ]:
class SenML_JSON_Serializer:
  def serialize(self, measurements: List[Measurement], bn_value: str, n_value: str) -> str:
    output = []
    values = [m.value for m in measurements]
    timestamps = [m.timestamp for m in measurements]
    data_size = len(values)
    index = data_size - 1
    base_time = timestamps[index]
    output.append({
      'bn': bn_value,
      'n': n_value,
      'v': values[index],
      'bt': timestamps[index],
      't': 0
    })
    index -= 1
    while index > -1:
      output.append({
        'n': n_value,
        'v': values[index],
        't': timestamps[index] - base_time,
      })
      index -= 1
    return json.dumps(output)

senml_json_serializer = SenML_JSON_Serializer()
print(senml_json_serializer.serialize(measurements, '/72/', '1/2'))

[{"bn": "/72/", "n": "1/2", "v": 2, "bt": 1200, "t": 0}, {"n": "1/2", "v": 3, "t": -100}, {"n": "1/2", "v": 5, "t": -200}]
